In [15]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from mongocrud import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "xj2097plaf164bv"
port = 37104
shelter = AnimalShelter(username, password, port)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dcc.RadioItems(
        id='radio-id',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain Rescue', 'value': 'mountain'},
            {'label': 'Disaster Rescue', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset'
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        style_data_conditional=[
            {
                'if': {'row_index': 'even'},
                'backgroundColor': 'rgb(200, 200, 200)'
            }
        ],
        row_selectable='single',
        page_size=10
    ),
    html.Br(),
     html.Hr(),
    html.Div(style={'display':'flex'}, children=[
     html.Div(id='graph-id'),
      html.Div(
            id='map-id',
            children=[dl.Map(style={'width': '500px', 'height': '350px'}, center=[30.75,-97.48], zoom=10,
                             children=[dl.TileLayer(id="base-layer-id")])],
            className='col s12 m6'
            )
    ]),
    html.Div([
        html.Img(src='https://learn.snhu.edu/content/enforced/851421-CS-340-T1035-OL-TRAD-UG.21EW1/course_documents/Grazioso%20Salvare%20Logo.png?_&d2lSessionVal=MoLJ81dpDRAGzansFcjGYiWxq&ou=851421',
                 height=200),
        html.H1('Harrison Bergeron', id="unique_identifier")
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "selected_rows")])
def update_map(selected_rows):
    #dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '700px', 'height': '350px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[df.iloc[selected_rows[0],13],df.iloc[selected_rows[0],14]], children=[
                dl.Tooltip(df.iloc[selected_rows[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(df.iloc[selected_rows[0],9])
                ])
            ])
        ])
    ]

def update_query(value):
    if value == 'water':
        return pd.DataFrame.from_records(shelter.read(
            {'$and': [
                {'$or': [
                    {'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesapeake Bay Retriever'},
                    {'breed': 'Newfoundland'}
                ]},
                {'sex_upon_outcome': 'Intact Female'},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
            }
        ))
    elif value == 'mountain':
        return pd.DataFrame.from_records(shelter.read(
            {'$and': [
                {'$or': [
                    {'breed': 'German Shepherd'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
            }
        ))
    elif value == 'disaster':
        return pd.DataFrame.from_records(shelter.read(
            {'$and': [
                {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
            ]
            }
        ))
    elif value == 'reset':
        return pd.DataFrame.from_records(shelter.read({}))

@app.callback(
    Output('graph-id', 'children'),
    [Input('radio-id', 'value')]
)
def update_graph(value):
    df = update_query(value)
    fig = px.pie(df, names='breed')
    return dcc.Graph(figure=fig)

@app.callback(
    Output('datatable-id', 'data'),
    [Input('radio-id', 'value')]
)
def update_dt(value):
    df = update_query(value)
    
    return df.to_dict('records')

app